# openEO use case for floods

### Connect and authenticate to openEO

In [ ]:
import openeo
from openeo.processes import *

# backend = "openeo.cloud"
backend = "https://openeo-dev.eodc.eu/openeo/1.1.0/"
conn = openeo.connect(backend)
conn = conn.authenticate_oidc()

### Describe the GFM collection

In [ ]:
conn.describe_collection("GFM")

### Sum of flooded_pixel over time

In [ ]:
spatial_extent  = {'west': 67.5, 'east': 70, 'south': 24.5, 'north': 26}
temporal_extent = ["2022-09-01", "2022-10-01"] 
collection      = 'GFM'

gfm_data = conn.load_collection(
    collection, 
    spatial_extent=spatial_extent, 
    temporal_extent=temporal_extent, 
    bands = ["flood_extent"]
)
gfm_sum = gfm_data.reduce_dimension(dimension="time", reducer=sum)

gfm_sum_tiff = gfm_sum.save_result(format="GTiff", options={"tile_grid": "equi7", "dtype": "int16"})

In [ ]:
job = gfm_sum_tiff.create_job(title = "UC11")
jobId = job.job_id
job.start_job()

In [ ]:
job.status()

In [ ]:
results = job.get_results()
metadata = results.get_metadata()
results.download_files("./gfm/flood_extent/")

In [ ]:
# really basic test to see if the results are actually as expected
import os
import xarray as xr
import matplotlib.pyplot as plt

path = "./gfm/flood_extent/"
files = [path + file for file in os.listdir(path) if file.startswith("AS020") and "20220901" in file]
D = xr.open_mfdataset(files)
d = D.flood_extent
print("Maximum value: ", d.max().values)
print("Minimum value: ", d.min().values)
display(d)
plt.figure(figsize=(10,5))
plt.title(f"Flood extent")
# only plot a part of the data
plt.imshow(d[5000:6000,5000:6000], cmap="Blues")
plt.colorbar()

### Reference water mask reduced over time

In [ ]:
spatial_extent  = {'west': 67.5, 'east': 70, 'south': 24.5, 'north': 26}
temporal_extent = ["2022-09-01", "2022-10-01"] 
collection      = 'GFM'

gfm_data = conn.load_collection(
    collection, 
    spatial_extent=spatial_extent, 
    temporal_extent=temporal_extent, 
    bands = ["refwater"]
)
gfm_sum = gfm_data.reduce_dimension(dimension="time", reducer=any)

gfm_sum_nc = gfm_sum.save_result(format="NetCDF", options={"tile_grid": "equi7", "dtype": "int16"}) 

In [ ]:
job = gfm_sum_nc.create_job(title = "UC11")
jobId = job.job_id
job.start_job()

In [ ]:
job.status()

In [ ]:
results = job.get_results()
metadata = results.get_metadata()
results.download_files("./gfm/refwater/")

In [ ]:
import os
import xarray as xr
import matplotlib.pyplot as plt

path = "./gfm/refwater/"
files = [path + file for file in os.listdir(path) if file.startswith("AS") and "20220901" in file]
D = xr.open_mfdataset(files)
d = D.refwater
display(d)
plt.figure(figsize=(10,5))
plt.title(f"Refwater")
# only plot a part of the data
plt.imshow(d[5000:6000,5000:6000], cmap="Blues")
plt.colorbar()

### Observed water (flood_extent + refwater)

In [ ]:
spatial_extent  = {'west': 67.5, 'east': 70, 'south': 24.5, 'north': 26}
temporal_extent = ["2022-09-01", "2022-10-01"] 
collection      = 'GFM'

gfm_data = conn.load_collection(
    collection, 
    spatial_extent=spatial_extent, 
    temporal_extent=temporal_extent, 
    bands = ["flood_extent", "refwater"]
)

# retrieve all pixels which have been detected as water during the given period
# -> observed water
observed_water = gfm_data.reduce_dimension(dimension="bands", reducer=any).reduce_dimension(dimension="time", reducer=any)

# Save the result in Equi7Grid and as GeoTiff
observed_water_tif = observed_water.save_result(format="GTiff", options={"tile_grid": "equi7", "dtype": "int16"})

In [ ]:
job = gfm_sum_nc.create_job(title = "UC11")
jobId = job.job_id
job.start_job()

In [ ]:
job.status()

In [ ]:
results = job.get_results()
metadata = results.get_metadata()
results.download_files("./gfm/observed_water/")

In [ ]:
import os
import xarray as xr
import matplotlib.pyplot as plt

path = "./gfm/observed_flood/"
files = [path + file for file in os.listdir(path) if file.startswith("AS") and file.endswith(".tif")]
# we expect only 0 and 1 -> bool
data = xr.open_mfdataset(files).astype("bool")
d = data.band_data
display(d)
plt.figure(figsize=(10,5))
plt.title("Observed water")
plt.imshow(d[0][5000:6000,5000:6000], cmap="Blues")
plt.colorbar()